# Projet 4 Bernadoy Corentin

Le but de ce projet est de créer une balise html `<a>` autour des textes de référencés dans les différents arrêtés municipaux mis à notre disposition.

## Imports

In [1]:
import os
import bs4
import regex as re

## Stratégie et étude du jeu de donnée

Pour une première approche, nous allons tenter d'utiliser un maximum les expressions régulières. En effet, il s'agit d'une approche simple et si elle réussit, permet de consitituer un premier jeu de données labélisées pour une étude par machine learning. 

Tout d'abord on établit la structure que l'on veut suivre : on va extraire les textes présents dans les balises via Beautifulsoup, recherche la présence de textes de lois dans ces textes extraits, les encadrer par une balise `<a>` puis on va sauvegarder le fichier html modifié dans un fichier séparé.

### Création des dossier de dépôt des résultats

On commence par créer les dossier de résultats dans l'arborescence actuelle

In [2]:
#Création des fichiers de résultat
root = "./data/"
os.makedirs(root + "results/", exist_ok= True) # créer un dir à côté du dir data
sub_data_dirs = next(os.walk(root + 'data/'))[1] # fait la liste des dossiers à créer
for sub_data_dir in sub_data_dirs:
    os.makedirs(root + "results/" + sub_data_dir, exist_ok= True) #réplique l'arborescence du dossier data

### Fonctions utiles

Il nous faut trois fonctions pour établir la structure : une première fonction `find_regex` pour évaluer les matchs avec les regex et déterminer leur position dans le texte, une fonction `rewritting` pour réécrire le texte avec les balises et enfin une fonction  `pipeline` qui itère les deux premières sur chaque texte de chaque fichier et qui sauvegarde les résultats

On écrit d'abord la list des regex que l'on va utiliser pour détecter les références à des textes de lois (plus de détails sur comment établir les regexs dans la partie suivante)

In [3]:
regex_list = [
    # regex pour les directives
    r"directive[\sA-Za-zÀ-ÿ]+(?:\s+n°|\s+no)?+\d{4}/\d+/[A-Z]{2}",
    # regex pour les décrets
    r"[Dd]écret(?:\s+n°|\s+no)?\s*[./\d\-]+(?:\s+du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4})?|[Dd]écret\s+du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4}",
    # regex pour les articles du code de l'environnemnent
    r"article(.+?)[Cc]ode\s+de\s+l[’']environnement|[\sA-Za-zÀ-ÿ]{2}\s+[Cc]ode\s+de\s+l[’']environnement",
    # regex pour les lois
    r"\s[Ll]oi(?:\s+n°|\s+no)?\s*[\d\-\s]+(?:du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4})?",
    # regex pour les circulaires
    r"circulaire[\sA-Za-zÀ-ÿ]+(?:\s+n°|\s+no)?(?:[A-Z/\d\-]+)\s+(?:du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4})?|circulaire[\sA-Za-zÀ-ÿ/]+(?:du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4})",
    # regex les arrêtés pour 
    r"[Aa]rrêté[\sA-Aa-zÀ-ÿ]+(?:\s+n°|\s+no)\s*[\d\-]+|[Aa]rrêté[\sA-Aa-zÀ-ÿ]+du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4}"
]

On écrit les fonctions

In [4]:
def find_regex(regex_list, text):
    occurences = [] #On crée une liste pour récupérer les positions où insérer les balises
    for regex in regex_list:
        for occurence in re.finditer(regex, text):
            occurences.append((occurence.start(0),occurence.end(0)))# On récupère les positions où insérer les balises
    return((len(occurences) > 0), occurences)

In [5]:
def rewriting(soup, tag, occurences): # fonction de réécriture
    sorted_occurences = sorted(occurences) # on trie la liste pour insérer les balises une par une
    spliced_text = [] # on créé une liste qui permettra de découper le texte
    start = 0 # on indique le premier indice à sélectionner
    for occ in sorted_occurences:
        spliced_text.append(tag.text[start:occ[0]]) #on découpe le texte jusqu'à rencontrer une balise à insérer
        spliced_text.append(tag.text[occ[0]:occ[1]]) #on découpe le texte à baliser
        start = occ[1] #on repart de après la balise
    spliced_text.append(tag.text[start:]) # on ajoute le segment de la dernière balise à la fin
    tag.string = '' #on efface le texte présent pour le baliser avec le texte découper
    for i, substring in enumerate(spliced_text):
        if (i%2 == 0): # si le segment a un indice pair, il est hors de balises : on l'ajoute sans le modifier
            tag.append(substring)
        if (i%2 != 0):  #sinon : on l'ajoute en le balisant : il faut créer un nouveau tag 'a'
            a_tag = soup.new_tag('a')
            a_tag.string = substring
            tag.append(a_tag)

In [6]:
root = "./data/"
def pipeline(root = root): # on créer la fonction pour itérer sur tous les fichiers
    data_dirs = root + 'data/'  #dossier des données
    sub_data_dirs = next(os.walk(data_dirs))[1] #sous-dossiers des données
    res_dirs = root + 'results/' # dossier des résultats
    for sub_data_dir in sub_data_dirs: # pour chaque sous dossier
        file_names = next(os.walk(data_dirs + sub_data_dir + '/'))[2] #on récupère les noms des fichiers
        for file_name in file_names: # pour chaque fichier
            data_path = data_dirs + sub_data_dir + '/' + file_name #on récupère le path
            res_data_path = res_dirs + sub_data_dir + '/' + file_name[:-5] + '_results.html' #on récupère le path où copier les résultat
            with open(data_path, 'r') as f: # pour chaque fichier de donnée
                soup = bs4.BeautifulSoup(f, 'html')   # on transforme en beautiful soup
                whole_text = soup.find_all(['div', 'td', 'h2']) # on extrait le texte des balises où il peut être
                for tag in whole_text: #pour chaque tag de texte
                    if (not(tag.find('h1'))): # on exclue le div qui contient le titre, car ce dernier contient toujours le nom du texte présenté dans le document, ce qui n'est pas une référence externe
                        found, occurences = find_regex(regex_list, tag.text) # on cherche des regex
                        if found: # si on en trouve
                            rewriting(soup, tag, occurences) #on réécrit le texte

            with open(res_data_path, 'w', encoding= "utf-8") as f:#on ouvre et on copie les résultats
                    f.write(str(soup))

In [7]:
pipeline() # cellule d'exécution de pipeline

## Comment établir les regexs :

On écrit une fonction `get_example` qui permet de récupérer toutes les balises contenant un mot de texte de loi à référencer, ce qui nous permet de récupérer toutes les occurences d'un type de loi dans les documents et donc d'avoir une idée de quelle regexp utiliser. On sauvegarde ces textes dans un fichier text séparé afin de pouvoir l'étudier plus facilement.

In [8]:
#### ne pas supprimer : permet de savoir quel format peut avoir chaque texte de loi
def get_examples(Loi, name):
    root = "./data/"
    data_dirs = root + 'data/'
    sub_data_dirs = next(os.walk(data_dirs))[1]
    res_dirs = root + 'results/'
    examples = [] # on récupère le texte des exemples dans une liste
    for sub_data_dir in sub_data_dirs:
        file_names = next(os.walk(data_dirs + sub_data_dir + '/'))[2]
        for file_name in file_names:
            data_path = data_dirs + sub_data_dir + '/' + file_name
            with open(data_path, 'r') as file_content:
                soup = bs4.BeautifulSoup(file_content, 'html')   
                whole_text = soup.find_all(['div', 'td', 'h2'])
                for tag in whole_text:
                    if(find_regex(Loi, tag.text)[0]):
                        examples.append(tag.text)

    with open(f"example_{name}.txt", 'w') as f: # on envoie les résultats dans un fichier texte
        for example in examples:
            f.write(f"{example}\n")

In [9]:
get_examples([r"[Aa]rrêté"], "Arrete") # exemple pour les arrêtés

À l'aide des documents d'exemple, on peut déterminer un pattern chaque texte de loi, qui permet de l'identifier au mieux:
- Pour les directives, la structure est généralement la suivante : le mot 'directive' suivi optionnellement du mot 'européenne' optionnellement d'un numéro et d'un code d'identification sous le format '####/##/AA' avec # des chiffres et A des lettres. À noter qu'il peut y avoir plus que deux chiffres au centre du code. On décide donc comme regex une expression qui encapsule du mot directive à la fin du code, ce qui se traduit ici par : `r"directive[\sA-Za-zÀ-ÿ]+(?:\s+n°|\s+no)?+\d{4}/\d+/[A-Z]{2}"`.
- Pour les décrets, ils sont généralement de la forme suivante : ils commencent par le mot 'décret', avec parfois un D majuscule, puis il peut y avoir soit un numéro indiqué (écrit no ou n°) et contenant des chiffres, des tirets, des points ou des /, puis possède une date de la forme (JJ mois AAAA) avec J et A des chiffres, mois le mois en toute lettre. L'autre forme possible est avec la date directement après le mot décret, de la même forme que pour la première possibilité. Cela donne en regex : `r"[Dd]écret(?:\s+n°|\s+no)?\s*[\d\-/.]+(?:\s+du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4})?|[Dd]écret\s+du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4}"`
- Pour le code de l'environnement, on peut tenter de récupérer des références aux articles pour pouvoir extraire des références plus précises, dans ce cas, le pattern est : "article" + d'éventuels autres articles et des lettres et chiffres faisant référence à l'article, puis le code de l'environnement avec éventuellement un C majusule. Attention, deux types d'apostrophe sont présents. S'il n'y a pas d'article, on tente d'identifier directement la référence au code de l'environnemnet, ce qui donne en tout : `r"article(.+?)[Cc]ode\s+de\s+l[’']environnement|[\sA-Za-zÀ-ÿ]{2}\s+[Cc]ode\s+de\s+l[’']environnement"`
- Pour les lois, la logique est similaire que pour les décrets, avec optionnellement un numéro de loi ou une date écrite.  En regex : `r"\s[Ll]oi(?:\s+n°|\s+no)?\s*[\d\-\s]+(?:du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4})?"`
- Pour les circulaires, la logique est similaire au décret, avec quelques différences de format des numéros. On essaye également de capter d'éventuels mots entre les deux, qui qualifient la circulaire. En regex : `r"circulaire[\sA-Za-zÀ-ÿ]+(?:\s+n°|\s+no)?(?:[A-Z/\d\-]+)\s+(?:du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4})?|circulaire[\sA-Za-zÀ-ÿ/]+(?:du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4})"`
- Pour les arrêtés, la logique est identique à celle des circulaires, ce qui donne en regex : `r"[Aa]rrêté[\sA-Aa-zÀ-ÿ]+(?:\s+n°|\s+no)\s*[\d\-]+|[Aa]rrêté[\sA-Aa-zÀ-ÿ]+du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4}"`

## Comparaisons des fichiers de sorties pour analyse des résultats.

Afin d'avoir une idée plus générale que d'éplucher les fichiers, on peut comparer les fichiers de résultat et de data pour évaluer la précision de la méthode par regex. 

In [10]:
def compare(Lois, regex_list):
    root = "./data/"
    data_dirs = root + 'data/'
    sub_data_dirs = next(os.walk(data_dirs))[1]
    res_dirs = root + 'results/'
    original = [] # on récupère le texte des exemples dans une liste
    diff = []
    results = []
    for sub_data_dir in sub_data_dirs:
        file_names = next(os.walk(data_dirs + sub_data_dir + '/'))[2]
        for file_name in file_names:
            data_path = data_dirs + sub_data_dir + '/' + file_name
            res_data_path = res_dirs + sub_data_dir + '/' + file_name[:-5] + '_results.html'
            with open(data_path, 'r') as file_content:
                soup = bs4.BeautifulSoup(file_content, 'html')   
                whole_text = soup.find_all(['div', 'td', 'h2'])
                for tag in whole_text:
                    for Loi in Lois:
                        if(find_regex(Loi, tag.text)[0]):
                            original.append(tag.text)
                    for reg in regex_list:
                        if(find_regex(reg, tag.text)[0]):
                            diff.append(tag.text)

            with open(res_data_path, 'r') as file_content:
                soup = bs4.BeautifulSoup(file_content, 'html')   
                whole_text = soup.find_all(['a'])
                for tag in whole_text:
                    results.append(tag.text)
    return(original, results, diff)
                

La fonction précédente nous donne trois listes à étudier : 
- la liste original dans laquelle on stocke toute les divisions ayant une mention de texte de loi, de manière indisciminée
- la liste diff qui nous donne toutes les divisions ayant du texte capté par les regex plus tôt
- la liste results qui est itérée sur les résultats et capte leur texte

In [11]:
original, result, diff = compare([[r"\sloi\s",r"[Dd]écret", r"[Aa]rrêté", r"[cC]ode\sde\sl[’']environnement", r"circulaire", r"directive"]], [regex_list])

In [12]:
print("Nombre de divisions mentionnant un nom de texte de loi : ", len(original))
print("Nombre de divisions où au moins une balise a été créée : ", len(diff))
print("Nombre de divisions créées : ", len(result))

Nombre de divisions mentionnant un nom de texte de loi :  4291
Nombre de divisions où au moins une balise a été créée :  2498
Nombre de divisions créées :  3296


On nettoie toutes les lignes en doubles (captées par plusieurs regex) dans orignial et dans diff

In [13]:
original_cleaned = []

for st in original:
    if st not in original_cleaned:
        original_cleaned.append(st)

print(len(original_cleaned))

3861


In [14]:
diff_cleaned = []

for st in diff:
    if st not in diff_cleaned:
        diff_cleaned.append(st)

print(len(diff_cleaned))

2277


On crée unrec, une liste qui répertorie toutes les lignes non captées dans diff

In [15]:
unrec = []
for line in original_cleaned:
    if (line not in diff):
        unrec.append(line)

print(len(unrec))

1584


Examinons les textes non reconnus

In [16]:
unrec

["\n    VU** le projet d'arrêté porté le 24 mars 2020 à la connaissance du demandeur ;\n   ",
 '\n       La société B+T ENERGIE France Sas, dont le siège social est sis 7 avenue de Strasbourg - Parc des Colines - 68350 Bantzenheim, est autorisée, sous réserve du respect des prescriptions annexées au présent arrêté, à exploiter sur le territoire de la commune de Bantzenheim, les installations détaillées dans les articles suivants.\n      ',
 '\n       Les dispositions des articles ministériels existants relatifs aux prescriptions générales applicables aux installations classées soumises à déclaration sont applicables aux installations classées soumises à déclaration incluses dans l’établissement des lors que ces installations ne sont pas pas régies par le présent arrêté préfectoral d’autorisation.\n      ',
 "\n       Les installations citées à l'article 1.2.1 ci-dessus sont reportées avec leurs références sur le plan de situation de l'établissement annexé au présent arrêté.\n      ",
 

On remarque qu'une grande partie de ces textes semblent être liés au mot arrêté, souvent référencé dans "ce présent arrêté" ou autres. On refait donc l'expérience en supprimant arrêté dees regex :

In [17]:
regex_list_sans_arrete = regex_list.remove(r"[Aa]rrêté[\sA-Aa-zÀ-ÿ]+(?:\s+n°|\s+no)\s*[\d\-]+|[Aa]rrêté[\sA-Aa-zÀ-ÿ]+du\s+\d{1,2}(?:er)?\s+[a-zéû]+\s+\d{4}")
original, result, diff = compare([[r"\sloi\s",r"[Dd]écret", r"[cC]ode\sde\sl[’']environnement", r"circulaire", r"directive"]], [regex_list])

In [18]:
original_cleaned = []

for st in original:
    if st not in original_cleaned:
        original_cleaned.append(st)

print("Nombre de divisions uniques avec un nom de texte de loi ", len(original_cleaned))

diff_cleaned = []

for st in diff:
    if st not in diff_cleaned:
        diff_cleaned.append(st)

print("Nombre de divisions uniques avec au moins une regex récupérée ", len(diff_cleaned))

unrec = []
for line in original_cleaned:
    if (line not in diff):
        unrec.append(line)

print("Nombre de divisions uniques avec une regex sous bon format mais non détectée ", len(unrec))

Nombre de divisions uniques avec un nom de texte de loi  1284
Nombre de divisions uniques avec au moins une regex récupérée  1184
Nombre de divisions uniques avec une regex sous bon format mais non détectée  100


In [19]:
unrec

['\n       En cas d’émissions de vibrations mécaniques générantes pour la voisinage ainsi que pour la sécurité des biens ou des personnes, les points de contrôle, les valeurs des niveaux limites admissibles ainsi que la mesure des niveaux vibratoires émis seront déterminés suivant les spécifications des règles techniques annexées à la circulaire ministérielle n° 23 du 23 juillet 1986 relatives aux vibrations mécaniques émises dans l’environnement par les installations classées.\n      ',
 "\n     Le présent décret ne peut être déféré qu'un tribunal administratif :\n     \n\n       1° par les tiers, personnes physiques ou morales, les communes intéressées ou leurs groupements, en raison des inconvénients ou des dangers que le fonctionnement de l'installation présente pour les intérêts mentionnés aux articles L. 211-1 et L. 511-1 dans un délai d'un an à compter de la publication ou de l'affichage de la présente décision ;\n      \n\n       2° par les demandeurs ou exploitants, dans un dé

On peut voir ici les limites de l'approche regex : certains textes acceptables ne sont pas repérés.
Par manque de temps, une étude qualitative des textes balisés n'a pas abouti. De même, on aurait pu raffiner les regex un maximum afin de diminuer les problèmes.